In [1]:
import pandas as pd
import datetime
import time
import re
import collections
import warnings
import sklearn
import pickle
import numpy as np
import json
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import KFold,train_test_split



In [32]:
# load the data
onDrive= False
if onDrive == True:
    path= "/content/drive/MyDrive/School/IDC/RS/Project/data/"
    from google.colab import drive
    drive.mount('/content/drive')
else:
    path= "data\\"
movies_MD_df = pd.read_csv(path + "movies_metadata.csv")
credits_df = pd.read_csv(path + "credits.csv")
ratings_df = pd.read_csv(path + "ratings.csv")
links_df = pd.read_csv(path + "links.csv")

C:\Users\oferinbar\AppData\Local\Temp\ipykernel_19900\758899751.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_MD_df = pd.read_csv(path + "movies_metadata.csv")


In [34]:
# drop unnecessary columns
movies_MD_df = movies_MD_df.drop(
    columns=[
        "adult",
        "belongs_to_collection",
        "homepage",
        "poster_path",
        "production_companies",
        "production_countries",
        "spoken_languages",
        "status",
        "tagline",
        "video",
        "budget",
        "revenue",
        "vote_average",
        "popularity",
    ]
, errors="ignore")
# Remove rows which have NaN values
movies_MD_df = movies_MD_df.dropna()
credits_df = credits_df.dropna()
links_df = links_df.dropna()
# change the id data type to int
movies_MD_df["id"] = movies_MD_df["id"].astype(int)
# merge on the id column
movies_creds_df = movies_MD_df.merge(credits_df, on="id")
# print how many items we have
print("Number of items: ", len(movies_creds_df))
links_df["tmdbId"] = links_df["tmdbId"].fillna(method="backfill").astype("int64")
links_df["movieId"]= links_df["movieId"].astype("int64")
columnMappings= {}

Number of items:  44488


In [35]:
movies_creds_df = movies_creds_df.merge(links_df, left_on="id", right_on="tmdbId", how="left")
print("Number of items: ", len(movies_creds_df))
#  drop duplicate movieIds
movies_creds_df = movies_creds_df.drop_duplicates(subset=["movieId"])
movies_creds_df.head()


Number of items:  44618


,genres,id,imdb_id,original_language,original_title,overview,release_date,runtime,title,vote_count,cast,crew,movieId,imdbId,tmdbId
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,Toy Story,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1,114709,862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,Jumanji,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",2,113497,8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22,101.0,Grumpier Old Men,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",3,113228,15602
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,Waiting to Exhale,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",4,114885,31357
4,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10,106.0,Father of the Bride Part II,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",5,113041,11862


In [36]:
# data preparation inspired by https://www.kaggle.com/code/surenj/movielens-eda
df= movies_creds_df.copy(deep=True)
df["release_date"] = (pd.to_datetime(df["release_date"], yearfirst= True )  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
genreRegex = re.compile(r": '(.*?)'")
df["genres"] = df["genres"].apply(lambda x: ', '.join(genreRegex.findall(x)))
castRegex = re.compile(r"'name': '(.*?)'")
df["cast"] = df["cast"].apply(lambda x: ', '.join(castRegex.findall(x)))
#  fill df["cast"] with "Unknown" if it is empty
df["cast"] = df["cast"].apply(lambda x: "Unknown" if x == "" else x)
directorRegex = re.compile(r"'Director', 'name': '(.*?)'")
df["director"] = df["crew"].apply(lambda x: ', '.join(directorRegex.findall(x)))
# writingRegex= re.compile(r"'writing', 'name': '(.*?)'")
# df["writing"] = df["crew"].apply(lambda x: ', '.join(writingRegex.findall(x)))
ScreenplayRegex= re.compile(r"'Screenplay', 'name': '(.*?)'")
df["Screenplay"] = df["crew"].apply(lambda x: ', '.join(ScreenplayRegex.findall(x)))
producerRegex= re.compile(r"'Producer', 'name': '(.*?)'")
df["producer"] = df["crew"].apply(lambda x: ', '.join(producerRegex.findall(x)))
genreCols= df.genres.str.get_dummies(sep=', ') #Creating dummy variables for genres
columnMappings["genres"]= genreCols.columns.to_list()

In [37]:
#  find number of movies in ratings_df that are missing from movies_MD_df
movies_MD_df_ids = set(df["movieId"])
ratings_df_ids = set(ratings_df["movieId"])
print(f' count of movies in ratings_df that are missing from movies_MD_df: {len(ratings_df_ids - movies_MD_df_ids)}')
print(f' count of movies in movies_MD_df that are missing from ratings_df: {len(movies_MD_df_ids - ratings_df_ids)}')
print(f' count of movies that are in both ratings_df and movies_MD_df: {len(ratings_df_ids & movies_MD_df_ids)}')
# only keep movies that are in both ratings_df and movies_MD_df
ratings_df = ratings_df[ratings_df["movieId"].isin(movies_MD_df_ids)]
print(f' count of ratings after removing movies that are missing from movies_MD_df: {len(ratings_df)}')
# only keep movies that are in both ratings_df and movies_MD_df
df = df[df["movieId"].isin(ratings_df_ids)]
print(f' count of movies in movies_MD_df after removing movies that are missing from ratings_df: {len(df)}')
# set ratings higher than 3.5 as 1, the rest as 0
ratings_df["rating"] = ratings_df["rating"].apply(lambda x: 1 if x > 3.5 else 0)

 count of movies in ratings_df that are missing from movies_MD_df: 1408
 count of movies in movies_MD_df that are missing from ratings_df: 705
 count of movies that are in both ratings_df and movies_MD_df: 43707
 count of ratings after removing movies that are missing from movies_MD_df: 25976672
 count of movies in movies_MD_df after removing movies that are missing from ratings_df: 43707


In [38]:
# make a colun for each director- only keep the top x directors
directorValueCounts= df["director"].dropna().value_counts()
directors = directorValueCounts.index.to_list()
print('top directors are directors with more than 5 movies')
topDirectors = [director for director in directors if directorValueCounts[director] > 5]
print(f'found {len(topDirectors)} top directors')
directorCols= pd.get_dummies(df["director"])
directorCols= directorCols.loc[::,topDirectors].copy()
print(f'shape of directorCols: {directorCols.shape}')
columnMappings["director"]= topDirectors


top directors are directors with more than 5 movies
found 1521 top directors
shape of directorCols: (43707, 1521)


In [39]:
# make a colun for each cast member- only keep the top 5 cast members per movie
cast= df["cast"].apply(lambda x: x.split(", ")[:5]).to_list()
castFlatList= [item for sublist in cast for item in sublist]
# keep only the top x cast members
castValueCounts= pd.Series(castFlatList).value_counts()
print('top cast members are cast members with more than 10 movies')
topCast = castValueCounts[castValueCounts > 10].index.to_list()
topCast.remove("Unknown")
print(f'found {len(topCast)} top cast members')
castCols= df["cast"].apply(lambda x: x.split(", ")[:5])
castCols= castCols.apply(pd.Series).stack(dropna=False)
castCols= castCols.apply(lambda x: x if x in topCast else "Other")
castCols= pd.get_dummies(castCols.apply(pd.Series).stack(), sparse=False).sum(level=0)
castCols= castCols.drop("Other", axis=1)
print(f'shape of castCols: {castCols.shape}')
columnMappings["cast"]= castCols.columns.to_list()


top cast members are cast members with more than 10 movies
found 3267 top cast members
shape of castCols: (43707, 3267)


In [40]:
languageCols= pd.get_dummies(df["original_language"])
print(f'shape of languageCols: {languageCols.shape}')
columnMappings["original_language"]= languageCols.columns.to_list()

shape of languageCols: (43707, 89)


In [41]:
dfCols2Keep= ['id', 'imdb_id','original_title',
       'overview', 'release_date', 'runtime', 'title', 'movieId', 'imdbId', 'tmdbId']
df= df[dfCols2Keep].copy()
columnMappings["dfCols"]= dfCols2Keep

In [57]:
# filter sparse movies, inspired by https://www.kaggle.com/code/morrisb/how-to-recommend-anything-deep-recommender
min_movie_ratings = 100
filtered_movies = (ratings_df.movieId.value_counts()>min_movie_ratings)
filtered_movies = filtered_movies[filtered_movies].index.tolist()

# Filter sparse users
min_user_ratings = 33
filtered_users = (ratings_df.userId.value_counts()>min_user_ratings)
filtered_users = filtered_users[filtered_users].index.tolist()

# Actual filtering
ratings_df2= ratings_df[(ratings_df.movieId.isin(filtered_movies)) & (ratings_df.userId.isin(filtered_users))]
print('Shape User-Ratings unfiltered:\t{}'.format(ratings_df.shape))
#  print number of movies and users
print(f'number of movies in ratings_df: {len(ratings_df["movieId"].unique())}')
print(f'number of users in ratings_df: {len(ratings_df["userId"].unique())}')
print(f'number of movies in ratings_df2: {len(ratings_df2["movieId"].unique())}')
print(f'number of users in ratings_df2: {len(ratings_df2["userId"].unique())}')
print('Shape User-Ratings filtered:\t{}'.format(ratings_df2.shape))

Shape User-Ratings unfiltered:	(25976672, 4)
number of movies in ratings_df: 43707
number of users in ratings_df: 270882
number of movies in ratings_df2: 9903
number of users in ratings_df2: 127924
Shape User-Ratings filtered:	(23386512, 4)


In [69]:
# randomly asign filtered_users to test and train sets
train_users, test_users = train_test_split(ratings_df2.userId.value_counts().index, test_size=0.2, random_state=42)
print(f'shapes of train_users, test_users: {train_users.shape, test_users.shape}')
#  dump train_users and test_users to disk
with open(path2Keep +'train_users.pkl', 'wb') as f:
    pickle.dump(train_users, f)
with open(path2Keep +'test_users.pkl', 'wb') as f:
    pickle.dump(test_users, f)
# randomly asign filtered_movies to test and train sets    
train_movies, test_movies = train_test_split(ratings_df2.movieId.value_counts().index, test_size=0.1, random_state=42)
#  dump train_movies and test_movies to disk
print(f'shapes of train_movies, test_movies: {train_movies.shape, test_movies.shape}')
with open(path2Keep +'train_movies.pkl', 'wb') as f:
    pickle.dump(train_movies, f)
with open(path2Keep +'test_movies.pkl', 'wb') as f:
    pickle.dump(test_movies, f)

shapes of train_users, test_users: ((102339,), (25585,))
shapes of train_movies, test_movies: ((8912,), (991,))


In [59]:
#  get the last rating for each user in the test set
lastX= 3
lastXRatingsIndex= ratings_df2.groupby('userId')['timestamp'].nlargest(lastX)
#  get the ratings from ratings_df2 that are in lastXRatings
lastXRatings= ratings_df2[ratings_df2.index.isin(lastXRatingsIndex.index.get_level_values(1))]
# last2Ratings.head()

In [60]:
# sort last2Ratings by userId and timestamp
lastXRatings= lastXRatings.sort_values(by=['userId', 'timestamp'])
print("last 5 ratings of user #4 : ")
print(ratings_df2[ratings_df2.userId == 4].sort_values(by='timestamp', ascending=False).head(5))
print("derived last X ratings of user #4:")
print(lastXRatings[lastXRatings.userId == 4].sort_values(by='timestamp', ascending=False))
#  get all ratings from ratings_df2 that are not in the lastXRatings
beforelastXRatings= ratings_df2[~ratings_df2.index.isin(lastXRatingsIndex.index.get_level_values(1))]
print("the items before the last X: ")
print(beforelastXRatings[beforelastXRatings.userId == 4].sort_values(by='timestamp', ascending=False).head(3))
print(f'shape of beforelastXRatings: {beforelastXRatings.shape}')
print(f'shape of lastXRatings: {lastXRatings.shape}')

last 5 ratings of user #4 : 
     userId  movieId  rating   timestamp
76        4     2338       0  1042674886
66        4     1422       1  1042674861
70        4     1644       0  1042674845
117       4     4238       1  1042674819
61        4      648       1  1042674800
derived last X ratings of user #4:
    userId  movieId  rating   timestamp
76       4     2338       0  1042674886
66       4     1422       1  1042674861
70       4     1644       0  1042674845
the items before the last X: 
     userId  movieId  rating   timestamp
117       4     4238       1  1042674819
61        4      648       1  1042674800
68        4     1597       0  1042674787
shape of beforelastXRatings: (23002740, 4)
shape of lastXRatings: (383772, 4)


In [65]:
#  pickle the last2Ratings and beforelast2Ratings to path2keep
# path2Keep= r"data2Keep\\" # path to save the data to
if onDrive == True:
    path2Keep= path.replace("/data", "") +  r"data2Keep/" # path to save the data to
else:
    path2Keep= r"data2Keep\\"

dfS= [df, genreCols, directorCols, castCols, languageCols]
movies_df= pd.concat(dfS, axis=1)
# only keep movies that are in the ratings_df2
movies_df= movies_df[movies_df.movieId.isin(ratings_df2.movieId)]
print(f'shape of movies_df: {movies_df.shape}')
# dump movies_df  to pickle
movies_df.to_pickle(path2Keep + "movies_df.pkl", compression='gzip')
lastXRatings.to_pickle(path2Keep + "lastXRatings.pkl", compression='gzip')
beforelastXRatings.to_pickle(path2Keep + "beforelastXRatings.pkl", compression='gzip')
# dump the column mappings to json
with open(path2Keep + "columnMappings.json", "w") as f:
    json.dump(columnMappings, f)

shape of movies_df: (9903, 4907)


In [67]:
# check the attributes of 122886 Star Wars: The Force Awakens
stacked= movies_df[movies_df.movieId == 122886].stack()
stacked[stacked!=0]

26471  id                                                          140607.0
       imdb_id                                                    tt2488496
       original_title                          Star Wars: The Force Awakens
       overview           Thirty years after defeating the Galactic Empi...
       release_date                                            1450137600.0
       runtime                                                        136.0
       title                                   Star Wars: The Force Awakens
       movieId                                                     122886.0
       imdbId                                                     2488496.0
       tmdbId                                                      140607.0
       Action                                                             1
       Adventure                                                          1
       Fantasy                                                            1
       Scien